# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng['Original Language'] = 'English'
    df_eng.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr['Original Language'] = 'French'
    df_fr.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    df_es = pd.read_csv("data/movie_reviews_sp.csv")
    df_es['Original Language'] = 'Spanish'
    df_es.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    
    df = pd.concat([df_eng, df_fr, df_es], axis=0)
    return df.reset_index(drop = True)

df = preprocess_data()

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [3]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    
    # encode the text using the tokenizer
    #inputs = None

    # generate the translation using the model
    outputs = model.generate(**tokenizer(text, return_tensors="pt", padding=True))

    # decode the generated output and return the translated text
    decoded = [tokenizer.decode(t, skip_special_tokens=True) for t in outputs]
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df['Original Language'] == 'French']['Review'].to_list()
fr_reviews_en = translate(fr_reviews, fr_en_model , fr_en_tokenizer)

# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'] == 'French']['Synopsis'].to_list()
fr_synopsis_en = translate(fr_synopsis, fr_en_model, fr_en_tokenizer)

# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'] == 'Spanish']['Review'].to_list()
es_reviews_en = translate(es_reviews, es_en_model , es_en_tokenizer)

# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'] == 'Spanish']['Synopsis'].to_list()
es_synopsis_en = translate(es_synopsis, es_en_model , es_en_tokenizer)

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [5]:
for i in range(10,20):
    
    rev = fr_reviews_en[i - 10]
    syn = fr_synopsis_en[i - 10]
    df.loc[i, 'Review'] = rev
    df.loc[i, 'Synopsis'] = syn

for i in range(20,len(df)):
    rev = es_reviews_en[i - 20]
    syn = es_synopsis_en[i - 20]
    df.loc[i, 'Review'] = rev
    df.loc[i, 'Synopsis'] = syn
    
    

In [6]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",French
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "sentiment-analysis"
sentiment_classifier = pipeline(task=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    preds = classifier(text)
    
    labels = [pred['label'] for pred in preds]
    
    return labels



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
sent_labels = analyze_sentiment(df['Review'].to_list(), sentiment_classifier)
df['Sentiment'] = sent_labels

In [11]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,POSITIVE
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish,POSITIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish,NEGATIVE
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French,POSITIVE
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,NEGATIVE
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French,NEGATIVE
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish,NEGATIVE
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,POSITIVE
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English,POSITIVE
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish,POSITIVE


In [10]:
# export the results to a .csv file
df.to_csv("./result/reviews_with_sentiment.csv", index=False)